In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
base_page_url = "https://www.somejobpage?pn="
page = requests.get(base_page_url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
max_page_element = soup.find('span', {'data-test': 'top-pagination-max-page-number'})
max_pages = int(max_page_element.text.strip())
print(max_pages)  #To check how many pages there are to scrape

1019


In [ ]:
base_page_url = "https://www.somejobpage"

# Superclass
class Page_And_Soup():
    def __init__(self, page_url):
        self.page = requests.get(page_url)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')

# Subclass
class Scrapping(Page_And_Soup):
    def __init__(self, page_url):
        # Initialisation of base class attributes
        super().__init__(page_url)

        # Creating a list of headers for the new DataFrame
        self.head = ["Title", "Link", "Company", "Region", "Additional_Info", "Current_page_url"]

        # Adding subsequent data from advertisements to a list
        self.Title = []
        self.Link = []
        self.Company = []
        self.Region = []
        self.Additional_Info = []
        self.Current_page_url = []

        # Extracting data for each job offer
        self.job_offers = self.soup.find_all('div', class_='tiles_c15wfuie')

        for offer in self.job_offers:
            self.offer_title = offer.find('h2', class_='tiles_h1yuv00i').text.strip()
            self.Title.append(self.offer_title)

            self.offer_link = offer.find('a', class_='core_n194fgoq')['href']
            self.Link.append(self.offer_link)

            self.company_name = offer.find('div', class_='tiles_ctlhhld').text.strip()
            self.Company.append(self.company_name)

            if offer.find('h4', class_='tiles_r1h1nge7'):
                self.region = offer.find('h4', class_='tiles_r1h1nge7').text
                self.Region.append(self.region)

            self.additional_info = []
            for info_item in offer.find_all('li', class_='tiles_iwlrcdk'):
                self.additional_info.append(info_item.text.strip())
            self.Additional_Info.append(self.additional_info)

            self.Current_page_url.append(page_url)

# Retrieving data for page 1
page_1 = Scrapping(base_page_url)

# Creating DataFrame from the first page
df = pd.DataFrame(list(zip(page_1.Title, page_1.Link, page_1.Company, page_1.Region, page_1.Additional_Info, page_1.Current_page_url)),
                  columns=page_1.head)

# Code for pages 2 to max_pages
for current_number in range(2, max_pages + 1):
    next_page_url = f"{base_page_url}?pn={current_number}"  # It loops through and thus I get the addresses of successive pages
    next_pages = Scrapping(next_page_url)  # Scraping successive pages

    # Collect data in lists
    data_to_concat = list(
        zip(next_pages.Title, next_pages.Link, next_pages.Company, next_pages.Region, next_pages.Additional_Info,
            next_pages.Current_page_url))

# Converting data to DataFrame
new_data_df = pd.DataFrame(data_to_concat, columns=page_1.head)

# Append data to the DataFrame
df = pd.concat([df, new_data_df], ignore_index=True)

# Saving to file
df.to_csv("/content/drive/MyDrive/job_offers.csv", index=False)
